Необходимо обучить модель для предсказания цены машины. База данных основана на объявленияъ с сайта "Юла".

Оцените качество работы созданной сети, определив средний процент ошибки на проверочной выборке. (Для этого потребуется привести предсказанные моделью значения к первоначальному диапазону цен. Это можно сделать с помощью следующего метода: predict_inverse = y_scaler.inverse_transfrom(predict).flatten() где predict - результат предсказания
модели). Затем подсчитайте ошибку на каждом примере тестовой выборки и суммарный процент ошибки. 

Рекомендации:
- В качестве ошибки рекомендуется использовать среднеквадратическую ошибку (mse).
- Метрику для данной задачи можно не использовать.
- Последний слой модели должен иметь 1 нейрон.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

# отрисовка изображений в ноутбуке, а не в файл или консоль
%matplotlib inline

In [2]:
from keras.models import Sequential  # НС прямого распространения
from keras.layers import Dense, Activation, Dropout, BatchNormalization  # основные слои
from keras import utils  # утилиты для to_categorical
from keras.preprocessing import image  # Для отрисовки изображения
from keras.optimizers import Adadelta, Adam  # Алгоритмы оптимизации для настройки скорости обучения

from sklearn.preprocessing import LabelEncoder, StandardScaler  # Функции для нормализации данных
from sklearn import preprocessing  # Пакет для предварительной обработки данных

In [ ]:
cars = pd.read_csv(r"\Neural networks\Data\cars_new.csv", sep=",")

In [4]:
cars = cars.dropna()

In [5]:
cars.head(3)

,mark,model,price,year,mileage,body,kpp,fuel,volume,power
0,kia,cerato,996000,2018,28000,седан,автомат,бензин,2.0,150.0
1,daewoo,nexia 1 поколение [2-й рестайлинг],140200,2012,60500,седан,механика,бензин,1.5,80.0
2,suzuki,jimny 3 поколение [рестайлинг],750000,2011,29000,внедорожник,автомат,бензин,1.3,85.0


In [6]:
print(cars.values.shape)

(70112, 10)


Напишем функцмю для преобразования столбца данных pandas DataFrame в OneHot

In [7]:
def labelsToOneHot(column):
    vocab, indexes = np.unique(cars[column], return_inverse=True)  # Получаем массив уникальных значений для солбца и тотже столбец в виде индексов
    oneHotData = utils.to_categorical(indexes, num_classes=len(vocab))  # Преобразуем индексы в onehot
    return oneHotData, vocab.tolist()  # Возвращаем индексы в OneHot и список уникальных значений

In [8]:
# Укажем столбцы с категориальными данными
labelData = ['mark', 'model', 'body', 'kpp', 'fuel']
vocabulary = []  # Список для списков всех уникальных значений. Его можно использовать для обратного преобразования из onehot в категориальные данные
oneHot = []  # Список для массивов onehot для всех столбцов
for column in labelData:
    oneHotData, vocab = labelsToOneHot(column)  # получим индексы и список уникальных значений для каждого столбца
    vocabulary.append(vocab)
    oneHot.append(oneHotData)
    oneHotArray = np.concatenate([i for i in oneHot], axis=1)

In [ ]:
oneHotArray

In [10]:
oneHotArray.shape

(70112, 3202)

In [11]:
for voc in vocabulary:
    print('Число уникальных элементов в', labelData[vocabulary.index(voc)], ':', len(voc))

Число уникальных элементов в mark : 21
Число уникальных элементов в model : 3156
Число уникальных элементов в body : 16
Число уникальных элементов в kpp : 4
Число уникальных элементов в fuel : 5


In [ ]:
print(vocabulary[0])

In [13]:
# Посчитаем количество всех автомобилей
lenmarks = len(vocabulary[0])

Получим из какой-нибудь строчки массива подмассив, принадлежащий к марке (порядок строчек сообтветствует порядку в DataFrame). Затем переведем его в индекс класса и сверим его с индексом в vocabulary (убедимся, что они идентичны)

In [14]:
np.argmax(oneHotArray[100, :lenmarks])
vocabulary[0][np.argmax(oneHotArray[100, :lenmarks])]

'mercedes-benz'

In [15]:
# Определим и запомним целевую переменную - цены
prices = np.array(cars['price'], dtype=np.float64)

In [16]:
# Запомним числовые параметры и нормализуем
years = preprocessing.scale(cars['year'])
mileages = preprocessing.scale(cars['mileage'])
volumes = preprocessing.scale(cars['volume'])
powers = preprocessing.scale(cars['power'])

In [17]:
years.mean()

np.float64(1.0392214499056326e-14)

In [18]:
mileages.std()

np.float64(1.0)

In [19]:
print(powers)

[ 0.22902799 -0.95629921 -0.87163298 ...  0.22902799 -0.75310026
  2.34568371]


Создадим обучающую выборку

In [20]:
x_train = []
y_train = []

for _id, car in enumerate(np.array(cars)):
    # добавляем цену в целевую переменную
    y_train.append(prices[_id])

    # В x_train добавляем все параметры. Категориальные переменные добавляем в виде ohe, числовые - как есть
    x_tr = oneHotArray[_id]
    x_tr = np.append(x_tr, years[_id])
    x_tr = np.append(x_tr, mileages[_id])
    x_tr = np.append(x_tr, volumes[_id])
    x_tr = np.append(x_tr, powers[_id])
    
    # добавим текущую строку в общий x_train
    x_train.append(x_tr)

# Превращаем лист в np.array после завершения цикла
x_train = np.array(x_train, dtype=np.float64)
y_train = np.array(y_train, dtype=np.float64)

In [21]:
print(x_train.shape)
print(y_train.shape)

(70112, 3206)
(70112,)


In [22]:
# Выведем один x_train
print(x_train[0, :20])
print(x_train[0,-20:])

[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[ 0.          1.          0.          0.          0.          0.
  0.          1.          0.          0.          0.          1.
  0.          0.          0.          0.          1.52001176 -1.4002026
  0.12284181  0.22902799]


In [23]:
# Выведем один y_train
print(y_train[:10])

[ 996000.  140200.  750000.  970000.  205000.  985000.  589000.  500000.
 1320000.  270000.]


Разделяем данные на тренировочную и тестовую выборки

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.1)

In [26]:
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1)).flatten()

In [27]:
y_train_scaled.shape

(63100,)

In [30]:
# Выведем базовый и нормированный y_train
print(y_train[:5])
print(y_train_scaled[:5])

[ 430000.  950000.  130000.  455000. 1450000.]
[-0.15641677  0.67288704 -0.63486128 -0.1165464   1.47029455]


In [31]:
x_train.shape

(63100, 3206)

Построим модель нейронной сети

In [ ]:
model_ula = Sequential()
model_ula.add(Dense(1000, activation='relu', input_shape=(x_train.shape[1],)))
model_ula.add(Dropout(0.2))
model_ula.add(Dense(100, activation='relu'))
model_ula.add(Dense(1, activation='linear'))

model_ula.compile(optimizer=Adam(learning_rate=0.0001), loss='mse')

# В обучающей выборке будет 50000 примеров
n_val = 50000
history = model_ula.fit(x_train[:n_val], y_train_scaled[:n_val],
                        batch_size=32,
                        epochs=15,
                        validation_data=(x_train[n_val:], y_train_scaled[n_val:]),
                        verbose=1)
# Отобразим графики ошибки обучения по обучающей выборке и по проверочной выборки на всех эпохах
plt.plot(history.history['loss'], label='Ошибка на обучающей выборке')
plt.plot(history.history['val_loss'], label='Ошибка на проверочной выборке')
plt.xlabel('Эпоха обучения')
plt.ylabel('MSE')
plt.legend()
plt.show()

In [33]:
# предсказываем проверочную выборку
prediction = model_ula.predict(x_test)

# Меняем масштаб обратно от нормированного к оригинальному
prediction = y_scaler.inverse_transform(prediction).flatten()

# Посчитаем среднюю цену, среднюю ошибку и средний процент ошибки
mean_delta = np.mean(abs(prediction - y_test))
mean_price = np.mean(y_test)

print('Средняя ошибка: ', round(mean_delta) )
print('Средняя цена: ', round(mean_price))
print('Сумарный процент ошибки :', round(100*mean_delta/mean_price), '%', sep='')

220/220 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Средняя ошибка:  67452
Средняя цена:  524536
Сумарный процент ошибки :13%


**Протестируем модель**

In [35]:
# Искусственные данные для тестирования
test_data = [
    ['kia', 'cerato', 2019, 25000, 'седан', 'автомат', 'бензин', 2.0, 150.0],
    ['daewoo', 'nexia', 2010, 120000, 'седан', 'механика', 'бензин', 1.5, 80.0],
    ['suzuki', 'jimny', 2020, 10000, 'внедорожник', 'автомат', 'бензин', 1.3, 85.0],
    ['bmw', 'x5', 2015, 80000, 'внедорожник', 'автомат', 'дизель', 3.0, 250.0]
]

# Создаем DataFrame
columns = ['mark', 'model', 'year', 'mileage', 'body', 'kpp', 'fuel', 'volume', 'power']
test_df = pd.DataFrame(test_data, columns=columns)

In [37]:
# Функция для преобразования категориальных признаков в one-hot encoding
def labelsToOneHot(column, vocabulary):
    # Создаем словарь для соответствия значений индексам
    value_to_index = {v: i for i, v in enumerate(vocabulary)}
    indexes = np.array([value_to_index.get(val, 0) for val in column])  # 0 для неизвестных значений
    oneHotData = utils.to_categorical(indexes, num_classes=len(vocabulary))
    return oneHotData

# Преобразуем категориальные признаки (используем сохраненные vocabulary)
labelData = ['mark', 'model', 'body', 'kpp', 'fuel']
oneHot_test = []

for i, column in enumerate(labelData):
    oneHotData = labelsToOneHot(test_df[column], vocabulary[i])
    oneHot_test.append(oneHotData)

# Объединяем one-hot encoded признаки
oneHotArray_test = np.concatenate([i for i in oneHot_test], axis=1)

In [39]:
# 1. Сохраняем scaler'ы для числовых признаков (делается один раз при обучении)
year_scaler = StandardScaler()
mileage_scaler = StandardScaler()
volume_scaler = StandardScaler()
power_scaler = StandardScaler()

# При обучении:
year_scaler.fit(cars[['year']])  # Обучаем scaler на тренировочных данных
mileage_scaler.fit(cars[['mileage']])
volume_scaler.fit(cars[['volume']])
power_scaler.fit(cars[['power']])

# Нормализуем числовые признаки тестовых данных
years_test = year_scaler.transform(test_df[['year']])
mileages_test = mileage_scaler.transform(test_df[['mileage']])
volumes_test = volume_scaler.transform(test_df[['volume']])
powers_test = power_scaler.transform(test_df[['power']])

In [40]:
# Объединяем все признаки
x_test_processed = np.column_stack([
    oneHotArray_test,
    years_test,
    mileages_test,
    volumes_test,
    powers_test
])

In [41]:
# Предсказание
predictions_scaled = model_ula.predict(x_test_processed)

# Обратное преобразование масштабированной целевой переменной
predictions = y_scaler.inverse_transform(predictions_scaled)

# Вывод результатов
for i, (data, pred) in enumerate(zip(test_data, predictions)):
    print(f"Автомобиль {i+1}:")
    print(f"  Марка: {data[0]}, Модель: {data[1]}, Год: {data[2]}, Пробег: {data[3]}")
    print(f"  Тип кузова: {data[4]}, КПП: {data[5]}, Топливо: {data[6]}")
    print(f"  Объем: {data[7]}, Мощность: {data[8]}")
    print(f"  Предсказанная цена: {pred[0]:.2f} руб.")
    print()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Автомобиль 1:
  Марка: kia, Модель: cerato, Год: 2019, Пробег: 25000
  Тип кузова: седан, КПП: автомат, Топливо: бензин
  Объем: 2.0, Мощность: 150.0
  Предсказанная цена: 1147663.38 руб.

Автомобиль 2:
  Марка: daewoo, Модель: nexia, Год: 2010, Пробег: 120000
  Тип кузова: седан, КПП: механика, Топливо: бензин
  Объем: 1.5, Мощность: 80.0
  Предсказанная цена: 119326.16 руб.

Автомобиль 3:
  Марка: suzuki, Модель: jimny, Год: 2020, Пробег: 10000
  Тип кузова: внедорожник, КПП: автомат, Топливо: бензин
  Объем: 1.3, Мощность: 85.0
  Предсказанная цена: 1419066.25 руб.

Автомобиль 4:
  Марка: bmw, Модель: x5, Год: 2015, Пробег: 80000
  Тип кузова: внедорожник, КПП: автомат, Топливо: дизель
  Объем: 3.0, Мощность: 250.0
  Предсказанная цена: 3885555.50 руб.



# Сохранение модели и вспомогательных объектов

In [ ]:
import joblib
import numpy as np
from keras.models import save_model

In [ ]:
# Сохраняем модель
save_model(model_ula, 'car_price_model.h5')

# Сохраняем scaler'ы для числовых признаков
joblib.dump(year_scaler, 'scalers/year_scaler.pkl')
joblib.dump(mileage_scaler, 'scalers/mileage_scaler.pkl')
joblib.dump(volume_scaler, 'scalers/volume_scaler.pkl')
joblib.dump(power_scaler, 'scalers/power_scaler.pkl')

# Сохраняем scaler для целевой переменной
joblib.dump(y_scaler, 'scalers/y_scaler.pkl')

In [ ]:
# Сохраняем vocabulary для категориальных признаков
vocabulary_dict = {
    'mark': vocabulary[0],
    'model': vocabulary[1],
    'body': vocabulary[2],
    'kpp': vocabulary[3],
    'fuel': vocabulary[4]
}
joblib.dump(vocabulary_dict, 'vocabulary.pkl')

In [ ]:
# Сохраняем порядок признаков (важно для сборки)
feature_order = [
    *[f'mark_{v}' for v in vocabulary[0]],
    *[f'model_{v}' for v in vocabulary[1]],
    *[f'body_{v}' for v in vocabulary[2]],
    *[f'kpp_{v}' for v in vocabulary[3]],
    *[f'fuel_{v}' for v in vocabulary[4]],
    'year', 'mileage', 'volume', 'power'
]
joblib.dump(feature_order, 'feature_order.pkl')

# Загрузка и использование для предсказаний

In [ ]:
import joblib
import numpy as np
import pandas as pd
from keras.models import load_model
from keras import utils

In [ ]:
# Загружаем все необходимые объекты
model = load_model('car_price_model.h5')
year_scaler = joblib.load('scalers/year_scaler.pkl')
mileage_scaler = joblib.load('scalers/mileage_scaler.pkl')
volume_scaler = joblib.load('scalers/volume_scaler.pkl')
power_scaler = joblib.load('scalers/power_scaler.pkl')
y_scaler = joblib.load('scalers/y_scaler.pkl')
vocabulary = joblib.load('vocabulary.pkl')
feature_order = joblib.load('feature_order.pkl')

def preprocess_input(new_data):
    """Функция для предобработки новых данных"""
    # Преобразуем в DataFrame если это еще не сделано
    if not isinstance(new_data, pd.DataFrame):
        new_df = pd.DataFrame([new_data], columns=['mark', 'model', 'year', 'mileage', 
                                                'body', 'kpp', 'fuel', 'volume', 'power'])
    else:
        new_df = new_data.copy()
    
    # One-Hot Encoding для категориальных признаков
    one_hot_encoded = []
    for col in ['mark', 'model', 'body', 'kpp', 'fuel']:
        vocab = vocabulary[col]
        value_to_idx = {v: i for i, v in enumerate(vocab)}
        indexes = np.array([value_to_idx.get(val, -1) for val in new_df[col]])
        one_hot = utils.to_categorical(indexes, num_classes=len(vocab))
        one_hot_encoded.append(one_hot)
    
    one_hot_array = np.concatenate(one_hot_encoded, axis=1)
    
    # Масштабирование числовых признаков
    years = year_scaler.transform(new_df[['year']])
    mileages = mileage_scaler.transform(new_df[['mileage']])
    volumes = volume_scaler.transform(new_df[['volume']])
    powers = power_scaler.transform(new_df[['power']])
    
    # Собираем все признаки вместе
    processed = np.column_stack([
        one_hot_array,
        years,
        mileages,
        volumes,
        powers
    ])
    
    return processed

def predict_price(car_data):
    """Функция для предсказания цены"""
    processed_data = preprocess_input(car_data)
    scaled_pred = model.predict(processed_data)
    prediction = y_scaler.inverse_transform(scaled_pred)
    return prediction[0][0]

In [ ]:
# Пример использования
new_car = {
    'mark': 'kia',
    'model': 'cerato',
    'year': 2020,
    'mileage': 15000,
    'body': 'седан',
    'kpp': 'автомат',
    'fuel': 'бензин',
    'volume': 2.0,
    'power': 150.0
}

In [ ]:
predicted_price = predict_price(new_car)
print(f"Предсказанная цена: {predicted_price:.2f} руб.")

**Структура папок проекта**

- │   car_price_model.h5 - # Сохраненная модель
- │   vocabulary.pkl  -  # Словари для категориальных признаков
- │   feature_order.pkl  -  # Порядок признаков
- │   predict.py - # Скрипт для предсказаний
- │   train.py  -  # Скрипт для обучения
- │
- └───scalers
    - │   year_scaler.pkl
    - │   mileage_scaler.pkl
    - │   volume_scaler.pkl
    - │   power_scaler.pkl
    - │   y_scaler.pkl